#### Install dependencies

In [1]:
%pip install -U openmim
!mim install mmcv-full

# install mmpose dependencies
%pip install -v -r requirements.txt

%pip install mmdet
%pip install mmpose
# install mmpose in develop mode
%pip install -v -e .

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.5/237.5 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 21.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
/usr/local/lib/python3.9/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Looking in links: https://download.openmmlab.com/mmcv/dist/cu116/torch1.12.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 MB 8.9 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.2/190.2 kB 24.7 MB/s eta 0:00:00
Using pip 22.2.2 from /usr/local/lib/python3.9/dist-packages/pip (python 3.9)
Ignoring dataclasses: markers 'python_version == "3.6"' don't match your environment
  Cloning https://github.com/svenkreiss/poseval.git to /tmp/pip-install-el8z7ojz/poseval_ea0a9e

In [2]:
import mmcv
from mmcv import Config, DictAction
import mmpose
import torch, torchvision
import time
import os
import shutil
import os.path as osp
import numpy as np
import pickle

import time
import warnings
import copy

from collections import OrderedDict, defaultdict
import torch.distributed as dist
from mmcv.runner import get_dist_info, init_dist, set_random_seed
from mmcv.utils import get_git_hash
from mmpose import __version__
from mmpose.apis import init_random_seed, train_model
from mmpose.models import build_posenet
from mmpose.utils import collect_env, get_root_logger, setup_multi_processes

from mmcv.cnn import fuse_conv_bn
from mmcv.parallel import MMDataParallel, MMDistributedDataParallel
from mmcv.runner import get_dist_info, init_dist, load_checkpoint

from mmpose.apis import multi_gpu_test, single_gpu_test
from mmpose.datasets import build_dataloader, build_dataset

try:
    from mmcv.runner import wrap_fp16_model
except ImportError:
    warnings.warn('auto_fp16 from mmpose will be deprecated from v0.15.0'
                  'Please install mmcv>=1.1.4')
    from mmpose.core import wrap_fp16_model
    
from matplotlib import pprint
from mmpose.apis import (inference_top_down_pose_model, init_pose_model,
                         vis_pose_result, process_mmdet_results)
from mmdet.apis import inference_detector, init_detector
from xtcocotools.coco import COCO

local_runtime = False
import cv2
from matplotlib import pyplot as plt
from mmpose.core.bbox.transforms import bbox_xywh2xyxy
from mmpose.datasets import DatasetInfo
import matplotlib
from mmpose.apis import (collect_multi_frames, get_track_id,
                         inference_top_down_pose_model, init_pose_model,
                         process_mmdet_results, vis_pose_tracking_result)

from mmpose.core import Smoother
from mmpose.apis import (collect_multi_frames, extract_pose_sequence,
                         get_track_id, inference_pose_lifter_model,
                         inference_top_down_pose_model, init_pose_model,
                         process_mmdet_results, vis_3d_pose_result)

from mmpose.core import Smoother
from mmpose.datasets import DatasetInfo
import pandas as pd
from PIL import Image
import os
import pandas as pd
import random
import math
import json


### 2D

#### Extract all AcinoSet 2D Labelled data: COCO format

In [ ]:
# NB: unzip "labelled_data.zip" to "/notebooks/data/acino/labelled" first

acino='/notebooks/data/acino'

img_ids=[os.path.splitext(img)[0] for img in os.listdir(acino+'/../ap10k/data/')]
img_ids.remove('.ipynb_checkpoints')
img_ids=[int(i) for i in img_ids]
img_no=max(img_ids)+1

base=pd.read_csv('base.csv')

for dir in os.listdir(acino+'/labelled'): #JamesFlick1 JamesFlick2 etc.
    print(dir)
    if os.path.isdir(acino+'/labelled/'+dir):
        
        df=pd.read_csv('labelled/'+dir+'/'+"CollectedData_UCT.csv")

        df.iloc[0,1:]=df.iloc[0,1:].astype(str) +'_'+ df.iloc[1,1:].astype(str)
        df.columns=df.iloc[0] # Make keypoint labels column headers
        df=df.iloc[2:,:]
        
        #Extract COCO keypoints (AP10K):
        #df=df.reindex(columns=(['bodyparts','l_eye_x','l_eye_y','r_eye_x','r_eye_y','nose_x','nose_y','neck_base_x','neck_base_y','tail_base_x','tail_base_y','l_shoulder_x','l_shoulder_y','l_front_knee_x','l_front_knee_y','l_front_paw_x','l_front_paw_y','r_shoulder_x','r_shoulder_y','r_front_knee_x','r_front_knee_y','r_front_paw_x','r_front_paw_y','l_hip_x','l_hip_y','l_back_knee_x','l_back_knee_y','l_back_paw_x','l_back_paw_y','r_hip_x','r_hip_y','r_back_knee_x','r_back_knee_y','r_back_paw_x','r_back_paw_y','tail2_x','tail2_y']))
        
        #Extract COCO keypoints (All Acino Keypoints):
        df=df.reindex(columns=(['bodyparts','l_eye_x','l_eye_y','r_eye_x','r_eye_y','nose_x','nose_y','neck_base_x','neck_base_y','tail_base_x','tail_base_y','l_shoulder_x','l_shoulder_y','l_front_knee_x','l_front_knee_y','l_front_paw_x','l_front_paw_y','r_shoulder_x','r_shoulder_y','r_front_knee_x','r_front_knee_y','r_front_paw_x','r_front_paw_y','l_hip_x','l_hip_y','l_back_knee_x','l_back_knee_y','l_back_paw_x','l_back_paw_y','r_hip_x','r_hip_y','r_back_knee_x','r_back_knee_y','r_back_paw_x','r_back_paw_y','tail2_x','tail2_y','tail1_x','tail1_y','r_front_ankle_x','r_front_ankle_y','l_front_ankle_x','l_front_ankle_y','r_back_ankle_x','r_back_ankle_y','l_back_ankle_x','l_back_ankle_y','spine_x','spine_y']))
        
        for i in range(0,len(df)): # Iterate over rows
            orig=df.iloc[i,0].split('/')[2]
            df.iloc[i,0]='as_'+str(img_no)+'.jpg'
            
            im=Image.open('data/acino/labelled/'+dir+'/'+orig)
            img_name='as_'+str(img_no)
            im.save('data/acino/data/'+img_name+'.jpg')
            
            #os.remove('labelled1/'+dir+'/'+orig)
            
            
            img_no+=1
        base=pd.concat([base,df[0:]],ignore_index=True)
               
base.to_csv("data/acino/annotations.csv",index=False)

In [ ]:
df = pd.read_csv('data/acino/annotations.csv')

if os.path.exists("data/acino/annotations/acino_all.json"):
    os.remove("data/acino/annotations/acino_all.json")
    
shutil.copyfile('data/acino/annotations/anns_empty_acino.json','data/acino/annotations/acino_all.json')

with open('data/acino/annotations/acino_all.json','r+') as f: # Save all annotation instances in one .json file
    anns_data=json.load(f)
    
    for i in range(0,len(df)):# Iterate over each image

        #Add image instance
        img_name=df.iloc[i,0]
        img_id=int(img_name[3:].split('.')[0])
        img=cv2.imread('data/acino/data/'+img_name)
        height = img.shape[0]
        width = img.shape[1]
        img_inst=dict(width=width,height=height,file_name=img_name,background=1,id=img_id)
        
        #Add annotation instance
        ann_id=img_id
        kp=[]
        num_kp=0

        # Initial bbox:
        x_min=width-1
        x_max=1
        y_min=height-1
        y_max=1

        for j in range(1,len(df.columns),2): # -12 for AP10k format
            if pd.isnull(df.iloc[i,j]): #NaN values
                kp.append(0)
                kp.append(0)
                kp.append(0)
                
            else:
                x=int(round(pd.to_numeric(df.iloc[i,j])))
                y=int(round(pd.to_numeric(df.iloc[i,j+1])))
                
                #Add annotations
                kp.append(x)
                kp.append(y)
                kp.append(2)
                num_kp+=1

                #Bbox
                if x<x_min:
                    x_min=x
                if x>x_max:
                    x_max=x
                if y>y_max:
                    y_max=y
                if y<y_min:
                    y_min=y
                    
        # Pad bbox with extra pixels:
        x_min=x_min-15
        y_min=y_min-15
        
        # BBox w, h
        w=max(1,(x_max-x_min))+30
        h=max(1,(y_max-y_min))+30
        a=w*h # Bbox area
        
        #Padded bounding box:
        bbox=[x_min,y_min,w,h] # xywh format NB! y_min==top left corner
        
        # Uncomment for AP10k format:
        
#         if kp[-1]==2: # If tip of tail != NaN
#             num_kp-=1 
            
#         del kp[-3:] # Remove tail_x, tail_y, visible
        
        
        ann_inst=dict(image_id=img_id,iscrowd=0,category_id=1,num_keypoints=num_kp,keypoints=kp,bbox=bbox,id=ann_id,area=a)
        anns_data['images'].append(img_inst)
        anns_data['annotations'].append(ann_inst)
    f.seek(0)
    json.dump(anns_data,f,indent=4)
f.close()

In [ ]:
%pip install echo1-coco-split

!coco-split --annotations_file data/acino/annotations/acino_all.json --valid_ratio 0.1 --test_ratio 0.2 --train_name data/acino/annotations/acino_train.json --valid_name data/acino/annotations/acino_val.json --test_name data/acino/annotations/acino_test.json --has_annotations

#### Train, test and demo 2D pose estimator

##### Config file:

In [3]:
# Config file:
cfg = Config.fromfile('configs/animal/2d_kpt_sview_rgb_img/topdown_heatmap/acino/res50_acino_256x256.py')
# For new configs:

# wrkdir = os.path.join('./work_dirs',
                                # os.path.splitext(os.path.basename('configs/animal/2d_kpt_sview_rgb_img/topdown_heatmap/acino/hrnet_w32_acino_256x256.py'))[0]+'_'+time.strftime('%d-%m_', time.localtime())+"ep"+str(cfg.total_epochs))

# if (os.path.isdir(wrkdir)):
#     cfg.
# else:
#     cfg.work_dir=wrkdir
cfg.work_dir='work_dirs/res50_acino_256x256_epoch150'    
# create work_dir
mmcv.mkdir_or_exist(cfg.work_dir)
                                  
# Config Parameters:
cfg.total_epochs=150

cfg.resume_from=None # the checkpoint file to resume from

cfg.gpu_ids=range(1)
cfg.workflow=[('train',1)]

cfg.log_name = time.strftime('%d-%m-', time.localtime())+"ep"+str(cfg.total_epochs)
cfg.log_file = os.path.join(cfg.work_dir, f'{cfg.log_name}.log')

##### Train model from cfg: 

In [33]:
import h5py
# %pip install tables
dlcpw=mmcv.load('data/acino_3d/2019_03_07/menya/flick/cam1dlc.pickle')
dlc='data/acino_3d/2019_03_07/menya/flick/cam1dlc.h5'
# with h5py.File(dlc,'r') as f:
#     print(f.keys())

dlc=pd.read_hdf(dlc)
print(dlcpw.keys())
print(dlcpw[189]['pose'])
len(dlc)
# dlc.iloc[189]

dict_keys(['stats', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 

576

In [ ]:
# Train from config file

autoscale_lr = False # automatically scale lr with the number of gpus
launcher = 'none' # Job launcher. ['none', 'pytorch', 'slurm', 'mpi']
deterministic = False # Whether to set deterministic options for CUDNN backend.
diff_seed = False # Whether or not set different seeds for different ranks
seed = 0 

if autoscale_lr:
    # apply the linear scaling rule (https://arxiv.org/abs/1706.02677)
    cfg.optimizer['lr'] = cfg.optimizer['lr'] * len(cfg.gpu_ids) / 8

if launcher == 'none':
    distributed = False
    if len(cfg.gpu_ids) > 1:
        warnings.warn(
            f'We treat {cfg.gpu_ids} as gpu-ids, and reset to '
            f'{cfg.gpu_ids[0:1]} as gpu-ids to avoid potential error in '
            'non-distribute training time.')
        cfg.gpu_ids = cfg.gpu_ids[0:1]
else:
    distributed = True
    init_dist(launcher, **cfg.dist_params)
    # re-set gpu_ids with distributed training mode
    _, world_size = get_dist_info()
    cfg.gpu_ids = range(world_size)

# set multi-process settings
setup_multi_processes(cfg)

# init the logger before other steps
logger = get_root_logger(log_file=cfg.log_file, log_level=cfg.log_level)

# init the meta dict to record some important information such as
# environment info and seed, which will be logged
meta = dict()

# log env info
env_info_dict = collect_env()
env_info = '\n'.join([(f'{k}: {v}') for k, v in env_info_dict.items()])
dash_line = '-' * 60 + '\n'
# logger.info('Environment info:\n' + dash_line + env_info + '\n' +
#             dash_line)
meta['env_info'] = env_info

# log some basic info
logger.info(f'Distributed training: {distributed}')
# logger.info(f'Config:\n{cfg.pretty_text}')

# set random seeds
seed = init_random_seed(seed)
seed = seed + dist.get_rank() if diff_seed else seed
logger.info(f'Set random seed to {seed}, '
            f'deterministic: {deterministic}')
set_random_seed(seed, deterministic=deterministic)
cfg.seed = seed
meta['seed'] = seed

# build dataset
train_set = [build_dataset(cfg.data.train)]

# build model
model = build_posenet(cfg.model)

if len(cfg.workflow) == 2:
    val_dataset = copy.deepcopy(cfg.data.val)
    val_dataset.pipeline = cfg.data.train.pipeline
    train_set.append(build_dataset(val_dataset))

if cfg.checkpoint_config is not None:
    # save mmpose version, config file content
    # checkpoints as meta data
    cfg.checkpoint_config.meta = dict(
        mmpose_version=__version__ + get_git_hash(digits=7),
        # config=cfg.pretty_text,
    )

timestamp = time.strftime('%d-%m-', time.localtime())+"ep"+str(cfg.total_epochs)
print("LINE")
# train model
train_model(
    model,
    train_set, 
    cfg, 
    distributed=distributed, 
    validate=True, 
    timestamp=timestamp,
    meta=meta)

/notebooks/mmpose/utils/setup_env.py:32: UserWarning: Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  warnings.warn(
/notebooks/mmpose/utils/setup_env.py:42: UserWarning: Setting MKL_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  warnings.warn(
2022-10-04 14:23:38,446 - mmpose - INFO - Distributed training: False
2022-10-04 14:23:38,451 - mmpose - INFO - Set random seed to 0, deterministic: False


loading annotations into memory...
Done (t=0.07s)
creating index...
index created!
=> num_images: 5346
=> load 5289 samples
load checkpoint from torchvision path: torchvision://resnet50


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias



...

2022-10-04 16:07:01,119 - mmpose - INFO - Epoch [111][1/83]	lr: 5.000e-04, eta: 0:36:45, time: 5.131, data_time: 4.706, memory: 8556, heatmap_loss: 0.0004, acc_pose: 0.9117, loss: 0.0004
2022-10-04 16:07:01,537 - mmpose - INFO - Epoch [111][2/83]	lr: 5.000e-04, eta: 0:36:45, time: 0.418, data_time: 0.016, memory: 8556, heatmap_loss: 0.0004, acc_pose: 0.9226, loss: 0.0004
2022-10-04 16:07:01,960 - mmpose - INFO - Epoch [111][3/83]	lr: 5.000e-04, eta: 0:36:44, time: 0.424, data_time: 0.017, memory: 8556, heatmap_loss: 0.0004, acc_pose: 0.9297, loss: 0.0004
2022-10-04 16:07:02,379 - mmpose - INFO - Epoch [111][4/83]	lr: 5.000e-04, eta: 0:36:43, time: 0.418, data_time: 0.011, memory: 8556, heatmap_loss: 0.0004, acc_pose: 0.9310, loss: 0.0004
2022-10-04 16:07:03,541 - mmpose - INFO - Epoch [111][5/83]	lr: 5.000e-04, eta: 0:36:43, time: 1.162, data_time: 0.754, memory: 8556, heatmap_loss: 0.0004, acc_pose: 0.9367, loss: 0.0004
2022-10-04 16:07:03,953 - mmpose - INFO - Epoch [111][6/83]	lr: 5

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 757/757, 58.8 task/s, elapsed: 13s, ETA:     0s

2022-10-04 16:16:29,982 - mmpose - INFO - Epoch(val) [120][24]	AP: 0.9900, AP .5: 0.9901, AP .75: 0.9901, AP (M): 0.9980, AP (L): 0.9900, AR: 0.9980, AR .5: 0.9987, AR .75: 0.9987, AR (M): 0.9989, AR (L): 0.9979


Loading and preparing results...
DONE (t=0.03s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *keypoints*
DONE (t=0.15s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets= 20 ] =  0.990
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets= 20 ] =  0.990
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets= 20 ] =  0.990
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets= 20 ] =  0.998
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets= 20 ] =  0.990
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 20 ] =  0.998
 Average Recall     (AR) @[ IoU=0.50      | area=   all | maxDets= 20 ] =  0.999
 Average Recall     (AR) @[ IoU=0.75      | area=   all | maxDets= 20 ] =  0.999
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets= 20 ] =  0.999
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= larg

2022-10-04 16:16:38,264 - mmpose - INFO - Epoch [121][1/83]	lr: 5.000e-04, eta: 0:27:36, time: 8.271, data_time: 7.859, memory: 8556, heatmap_loss: 0.0003, acc_pose: 0.9496, loss: 0.0003
2022-10-04 16:16:38,917 - mmpose - INFO - Epoch [121][2/83]	lr: 5.000e-04, eta: 0:27:35, time: 0.653, data_time: 0.253, memory: 8556, heatmap_loss: 0.0003, acc_pose: 0.9069, loss: 0.0003
2022-10-04 16:16:39,329 - mmpose - INFO - Epoch [121][3/83]	lr: 5.000e-04, eta: 0:27:35, time: 0.413, data_time: 0.011, memory: 8556, heatmap_loss: 0.0003, acc_pose: 0.9253, loss: 0.0003
2022-10-04 16:16:39,747 - mmpose - INFO - Epoch [121][4/83]	lr: 5.000e-04, eta: 0:27:34, time: 0.418, data_time: 0.013, memory: 8556, heatmap_loss: 0.0003, acc_pose: 0.9632, loss: 0.0003
2022-10-04 16:16:42,823 - mmpose - INFO - Epoch [121][5/83]	lr: 5.000e-04, eta: 0:27:34, time: 3.076, data_time: 2.675, memory: 8556, heatmap_loss: 0.0004, acc_pose: 0.9367, loss: 0.0004
2022-10-04 16:16:43,234 - mmpose - INFO - Epoch [121][6/83]	lr: 5

##### Test 2D pose estimator:

In [ ]:
launcher='none' #job launcher ['none', 'pytorch', 'slurm', 'mpi']
checkpoint=cfg.work_dir+'/latest.pth'
evaluate='mAP' #evaluation metric, which depends on the dataset, "mAP"
fuse_convbn=True
gpu_id=cfg.gpu_ids[0] #only applicable to non-distributed testing
gpu_collect=True #whether to use gpu to collect results
local_rank=0
tmpdir=None
    
# set multi-process settings
setup_multi_processes(cfg)

# set cudnn_benchmark
if cfg.get('cudnn_benchmark', False):
    torch.backends.cudnn.benchmark = True
cfg.model.pretrained = None
# cfg.data.test.test_mode = True

# init distributed env first, since logger depends on the dist info.
if launcher == 'none':
    distributed = False
else:
    distributed = True
    init_dist(launcher, **cfg.dist_params)

# build the dataloader
test_set = build_dataset(cfg.data.test, dict(test_mode=True))

# step 1: give default values and override (if exist) from cfg.data
loader_cfg = {
    **dict(seed=cfg.get('seed'), drop_last=False, dist=distributed),
    **({} if torch.__version__ != 'parrots' else dict(
           prefetch_num=2,
           pin_memory=False,
       )),
    **dict((k, cfg.data[k]) for k in [
               'seed',
               'prefetch_num',
               'pin_memory',
               'persistent_workers',
           ] if k in cfg.data)
}
# step2: cfg.data.test_dataloader has higher priority
test_loader_cfg = {
    **loader_cfg,
    **dict(shuffle=False, drop_last=False),
    **dict(workers_per_gpu=cfg.data.get('workers_per_gpu', 1)),
    **dict(samples_per_gpu=cfg.data.get('samples_per_gpu', 1)),
    **cfg.data.get('test_dataloader', {})
}

data_loader = build_dataloader(test_set, **test_loader_cfg)



# build the model and load checkpoint
model = build_posenet(cfg.model)
fp16_cfg = cfg.get('fp16', None)
if fp16_cfg is not None:
    wrap_fp16_model(model)

load_checkpoint(model, checkpoint, map_location='cpu')   
if fuse_convbn:
    model = fuse_conv_bn(model)
if not distributed:
    model = MMDataParallel(model, device_ids=[gpu_id])
    outputs = single_gpu_test(model, data_loader)
else:
    model = MMDistributedDataParallel(
        model.cuda(),
        device_ids=[torch.cuda.current_device()],
        broadcast_buffers=False)
    outputs = multi_gpu_test(model, data_loader, tmpdir,
                             gpu_collect)
    

rank, _ = get_dist_info()
eval_config = cfg.get('evaluation', {})
# eval_config = merge_configs(eval_config, dict(metric=evaluate))

out_file = cfg.work_dir+'/test_results.json'

if rank == 0:
    if out_file:
        print(f'\nwriting results to {out_file}') 
        mmcv.dump(outputs, out_file)

    results = test_set.evaluate(outputs, cfg.work_dir, **eval_config) # mmpose/datasets/datasets/animal/... evaluate()
    for k, v in sorted(results.items()):
        print(f'{k}: {v}')

##### 2D Estimator inference, analysis:

In [ ]:
# Test model:

pose_checkpoint = cfg.work_dir+'/latest.pth'
json_file_gt='data/acino/annotations/acino_all.json'

# with open('./work_dirs/hrnet_w32_acino-10k_256x256_epoch50/result_keypoints.json','r') as f: 
#     json_file_pred=json.load(f)
# f.close()

# print()

coco = COCO(json_file_gt)
# print(coco)
img_keys = list(coco.imgs.keys())
# print(img_keys)
# # initialize pose model
pose_model = init_pose_model(cfg, pose_checkpoint)
dataset = pose_model.cfg.data.test.type
dataset_info = pose_model.cfg.data.test.get('dataset_info', None)
dataset_info = DatasetInfo(dataset_info)
return_heatmap=True
output_layer_names=None

n=1 # Num imgs to plot
ntest=len(img_keys)
show= np.random.choice(ntest,n)

#GT:
for i in range(n):
    img_id=img_keys[show[i]]
    image=coco.loadImgs(img_id)[0]
    img_name=os.path.join('data/acino/data',image['file_name'])
    
    # pred=next(inst for inst in json_file_pred if inst['image_id']==img_id)
    
    
    ann_ids=coco.getAnnIds(img_id)
    # print(ann_ids)
    ann=coco.anns[ann_ids[0]]
    # print(ann)
    animal_bbox=[]
    gt_keypoints=[]
    
    for ann_id in ann_ids:
        animal={}  
        ann=coco.anns[ann_id]
        animal['bbox']=ann['bbox']
        animal_bbox.append(animal)
        
        gt={}
        x=ann['bbox'][0]
        y=ann['bbox'][1]
        w=ann['bbox'][2]
        h=ann['bbox'][3]
        gt['bbox']=[x,y,x+w,y+h]
        gt['keypoints']=np.array(ann['keypoints']).reshape(24,3)
        gt_keypoints.append(gt)
    
    # x=ann['bbox'][0]
    # y=ann['bbox'][1]
    # w=ann['bbox'][2]
    # h=ann['bbox'][3]
#     print(ann['bbox'])
    # bbox=[x,y,x+w,y+h]
    
#     print(ann['bbox'])

    pose_results, returned_outputs = inference_top_down_pose_model(
        pose_model,
        img_name,
        person_results=animal_bbox, # BBoxs required for top down
        bbox_thr=None,
        format='xywh',
        dataset=dataset,
        dataset_info=dataset_info,
        return_heatmap=True, #for now
        outputs=output_layer_names)
    
    out_file_pred = os.path.join('vis_result', f'vis_as{img_id}.jpg')
    out_file_gt = os.path.join('vis_result', f'vis_gt_as{img_id}.jpg')
    vis_result_pred=vis_pose_result(pose_model,img_name,pose_results,radius=3,thickness=1,out_file=out_file_pred)
    # vis_result_pred = cv2.resize(vis_result_pred, dsize=None, fx=1.5, fy=1.5)
    
    vis_result_gt=vis_pose_result(pose_model,img_name,gt_keypoints,radius=3,thickness=1,out_file=out_file_gt)
    # vis_result_gt = cv2.resize(vis_result_gt, dsize=None, fx=1.5, fy=1.5)
    # vis_result_gt = vis_result_gt

    plt.figure(figsize=(15,10))
    plt.imshow(vis_result_gt[y-25:y+h+25,x-25:x+w+25])
    plt.show()
    
    plt.figure(figsize=(15,10))
    plt.imshow(vis_result_pred[y-25:y+h+50,x-25:x+w+50])
    plt.show()
    
#     os.makedirs('vis_result', exist_ok=True)
#     out_file = os.path.join('vis_result', f'vis_as{image_id}.jpg')
#     vis_result=vis_pose_result(pose_model,image_name,pose_results,radius=3,thickness=1,out_file=out_file)
#     vis_result = cv2.resize(vis_result, dsize=None, fx=1.5, fy=1.5)
#     # plt.figure(figsize=(15,10))
#     # plt.imshow(vis_result)
#     # plt.show()
    


In [ ]:
!python tools/analysis/analyze_logs.py \
        plot_curve work_dirs/hrnet_w32_acino_256x256_epoch50/None.log.json \
        --keys acc_pose \
        --legend Accuracy \
        --out acino_accuracy.jpg

##### 2D Estimator demo with MMDet

In [ ]:
det_config='MMDet/configs/faster_rcnn_r50_fpn_2x_acino.py'
det_checkpoint='MMDet/epoch_4.pth'

pose_config='configs/animal/2d_kpt_sview_rgb_img/topdown_heatmap/acino/res50_acino_256x256.py'
pose_checkpoint='work_dirs/res50_acino_256x256_epoch150/latest.pth'

vid='menyacam3.mp4'#'../storage/cam3.mp4'
out_vid='work_dirs/res50_acino_256x256_epoch150/'
use_multi_frames=False
online=True
thickness=1
radius=4
smooth=True
smooth_filter_cfg='configs/_base_/filters/one_euro.py'
device='cuda:0'
bbox_thr=0.7
kpt_thr=0.5
use_oks_tracking=True
tracking_thr=0.3
euro=0
det_cat_id=1
show=False


print('Initializing model...')
det_model = init_detector(
    det_config, det_checkpoint, device=device.lower())
# build the pose model from a config file and a checkpoint file
pose_model = init_pose_model(
    pose_config, pose_checkpoint, device=device.lower())

dataset = pose_model.cfg.data['test']['type']
dataset_info = pose_model.cfg.data['test'].get('dataset_info', None)
if dataset_info is None:
    warnings.warn(
        'Please set `dataset_info` in the config.'
        'Check https://github.com/open-mmlab/mmpose/pull/663 for details.',
        DeprecationWarning)
else:
    dataset_info = DatasetInfo(dataset_info)
print(dataset_info.skeleton)
# read video
video = mmcv.VideoReader(vid)
assert video.opened, f'Failed to load video file {vid}'

if out_vid == '':
    save_out_video = False
else:
    os.makedirs(out_vid, exist_ok=True)
    save_out_video = True

if save_out_video:
    fps = video.fps
    size = (video.width, video.height)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    videoWriter = cv2.VideoWriter(
        os.path.join(out_vid,
                     f'vis_{os.path.basename(vid)}'), fourcc,
        fps, size)

# frame index offsets for inference, used in multi-frame inference setting
if use_multi_frames:
    assert 'frame_indices_test' in pose_model.cfg.data.test.data_cfg
    indices = pose_model.cfg.data.test.data_cfg['frame_indices_test']

# build pose smoother for temporal refinement
if euro:
    warnings.warn(
        'Argument --euro will be deprecated in the future. '
        'Please use --smooth to enable temporal smoothing, and '
        '--smooth-filter-cfg to set the filter config.',
        DeprecationWarning)
    smoother = Smoother(
        filter_cfg='configs/_base_/filters/one_euro.py', keypoint_dim=2)
elif smooth:
    smoother = Smoother(filter_cfg=smooth_filter_cfg, keypoint_dim=2)
else:
    smoother = None

# whether to return heatmap, optional
return_heatmap = False

# return the output of some desired layers,
# e.g. use ('backbone', ) to return backbone feature
output_layer_names = None

next_id = 0
pose_results = []
print('Running inference...')
for frame_id, cur_frame in enumerate(mmcv.track_iter_progress(video)):
    pose_results_last = pose_results

    # get the detection results of current frame
    # the resulting box is (x1, y1, x2, y2)
    mmdet_results = inference_detector(det_model, cur_frame)

    # keep the person class bounding boxes.
    person_results = process_mmdet_results(mmdet_results, det_cat_id)

    if use_multi_frames:
        frames = collect_multi_frames(video, frame_id, indices,
                                      online)

    # test a single image, with a list of bboxes.
    pose_results, _ = inference_top_down_pose_model(
        pose_model,
        frames if use_multi_frames else cur_frame,
        person_results,
        bbox_thr=bbox_thr,
        format='xyxy',
        dataset=dataset,
        dataset_info=dataset_info,
        return_heatmap=return_heatmap,
        outputs=output_layer_names)

    # get track id for each person instance
    pose_results, next_id = get_track_id(
        pose_results,
        pose_results_last,
        next_id,
        use_oks=use_oks_tracking,
        tracking_thr=tracking_thr,
        sigmas=dataset_info.sigmas)

    # post-process the pose results with smoother
    if smoother:
        pose_results = smoother.smooth(pose_results)

    # show the results
    vis_frame = vis_pose_tracking_result(
        pose_model,
        cur_frame,
        pose_results,
        radius=radius,
        thickness=thickness,
        dataset=dataset,
        dataset_info=dataset_info,
        kpt_score_thr=kpt_thr,
        show=False)

    if show:
        cv2.imshow('Frame', vis_frame)

    if save_out_video:
        videoWriter.write(vis_frame)

    if show and cv2.waitKey(1) & 0xFF == ord('q'):
        break

if save_out_video:
    videoWriter.release()

### 3D

#### Extract all AcinoSet 3D data

In [3]:
def add_cameras(n,calib_file,cameras_dict):
    calib_dict=mmcv.load(calib_file)
    
    for i in range(n):
        R=np.array(calib_dict['cameras'][i]['r'])
        T=np.array(calib_dict['cameras'][i]['t'])
        K=np.array(calib_dict['cameras'][i]['k'][:2])
        k=np.array(calib_dict['cameras'][i]['d'][:2])
        k=np.append(k,[k[1]/2],axis=0)
        p=np.array(calib_dict['cameras'][i]['d'][-2:])
        w=calib_dict['camera_resolution'][0]
        h=calib_dict['camera_resolution'][1]
        
        date=calib_file.split('/')[2]
        
        if 'top' in calib_file.split('/'):
            name=f'{date}_1_cam{i+1}'
            idx=f'{date}_1{i+1}'.replace('_','')
        elif 'bottom' in calib_file.split('/'):
            name=f'{date}_2_cam{i+1}'
            idx=f'{date}_2{i+1}'.replace('_','')
        else:
            assert calib_file.split('/')[2].split('_')[0]=='2019'
            name=f'{date}_cam{i+1}'
            idx=f'{date}_0{i+1}'.replace('_','')
            
        cameras_dict[idx]=dict(R=R,T=T,K=K,k=k,p=p,w=w,h=h,name=name)
        cameras_dict[idx]['id']=idx
        
    return cameras_dict

def get_cam(camera_id,cameras):
    for item in list(cameras.keys()):
        copy=item.replace('_','')
        copy=copy.replace('cam','0')
        if copy==camera_id:
            
            return cameras[item]
        else:
            raise Exception("Check camera dict")

def _get_pose_stats(kps):
    """Get statistic information `mean` and `std` of pose data.
    Args:
        kps (ndarray): keypoints in shape [..., K, C] where K and C is
            the keypoint category number and dimension.
    Returns:
        mean (ndarray): [K, C]
    """
    assert kps.ndim > 2
    K, C = kps.shape[-2:]
    kps = kps.reshape(-1, K, C)
    mean = kps.mean(axis=0)
    std = kps.std(axis=0)
    return mean, std

def get_dlc_kp2d(date,cheetah,action,cam):
    anns2d=pd.read_csv(f'data/acino_3d/{date}/{cheetah}/{action}/fte_pw/'+cam.split('.')[0]+'_fte.csv')
    anns2d.columns=anns2d.columns+anns2d.iloc[0] # Make keypoint labels column headers
    anns2d=anns2d.iloc[1:,1:]

    kp2d=[]
    
    for img in range(len(anns2d)):
        for kp in range(0,len(anns2d.columns),3):
            if math.isnan(float(anns2d.iloc[img,kp])): 
                kp2d.append(0)
                kp2d.append(0)
                kp2d.append(bool(0))
            else:
                kp2d.append(float(anns2d.iloc[img,kp]))           
                kp2d.append(float(anns2d.iloc[img,kp+1]))
                kp2d.append(bool(1))
                
    kp2d=np.array(kp2d,dtype=float).reshape(len(anns2d),20,3)
    # calculate bounding boxes,centers,scales
    scale_factor=1.2
    bboxes = np.stack([np.min(kp2d[:, :, 0], axis=1),np.min(kp2d[:, :, 1], axis=1),np.max(kp2d[:, :, 0], axis=1),np.max(kp2d[:, :, 1], axis=1)],axis=1)
    centers = np.stack([(bboxes[:, 0] + bboxes[:, 2]) / 2,(bboxes[:, 1] + bboxes[:, 3]) / 2], axis=1)
    scales = scale_factor * np.max(bboxes[:, 2:] - bboxes[:, :2], axis=1) / 200
    
    return kp2d,centers,scales


def get_anns_dlc(cam_id, cheetah, action, cam, date, start_frame, end_frame):
    ''' Extract frames from each camera and get 2D, 3D annotations'''
    
    #Save frames
    imgnames=[]
    video=mmcv.VideoReader(f'data/acino_3d/{date}/{cheetah}/{action}/{cam}')
    for frame_id, cur_frame in enumerate(mmcv.track_iter_progress(video)):
        if frame_id>=start_frame and frame_id<end_frame: 
            imgname=f'{cheetah}'+date.split('_')[1]+(date.split('_')[2])+f'_{action}.{cam_id}_{count}.jpg'
            mmcv.imwrite(video[frame_id],f'data/acino_3d/images/{imgname}')  # save frame as JPEG file      
            imgnames.append(imgname)
    imgnames=np.array(imgnames)
    
    #2D Anns
    anns2d=pd.read_csv(f'data/acino_3d/{date}/{cheetah}/{action}/fte_pw/'+cam.split('.')[0]+'_fte.csv')
    anns2d.columns=anns2d.columns+anns2d.iloc[0] # Make keypoint labels column headers
    anns2d=anns2d.iloc[1:,1:]
    kp2d=[]
    
    for img in range(len(anns2d)):
        for kp in range(0,len(anns2d.columns),3):
            if math.isnan(float(anns2d.iloc[img,kp])): 
                kp2d.append(0)
                kp2d.append(0)
                kp2d.append(bool(0))
            else:
                kp2d.append(float(anns2d.iloc[img,kp]))           
                kp2d.append(float(anns2d.iloc[img,kp+1]))
                kp2d.append(bool(1))
                
    kp2d=np.array(kp2d,dtype=float).reshape(len(anns2d),20,3)
    
    # calculate bounding boxes,centers,scales
    scale_factor=1.2
    bboxes = np.stack([np.min(kp2d[:, :, 0], axis=1),np.min(kp2d[:, :, 1], axis=1),np.max(kp2d[:, :, 0], axis=1),np.max(kp2d[:, :, 1], axis=1)],axis=1)
    centers = np.stack([(bboxes[:, 0] + bboxes[:, 2]) / 2,(bboxes[:, 1] + bboxes[:, 3]) / 2], axis=1)
    scales = scale_factor * np.max(bboxes[:, 2:] - bboxes[:, :2], axis=1) / 200
    
    #3D Anns
    anns3d=mmcv.load(f'data/acino_3d/{date}/{cheetah}/{action}/fte_pw/fte.pickle')
    kp3d=anns3d['positions']:
    kp3d = np.array(kp3d,dtype=float).reshape(len(anns3d['positions']),20,3)
    kp3d = np.concatenate([kp3d, np.ones((len(kp3d), 20, 1))],
                                axis=2)
    
    assert len(imgnames)==kp3d.shape[0]
    assert kp3d.shape[0]==kp2d.shape[0]
    
    return imgnames,centers,scales,kp2d,kp3d

def get_anns_res50(cam_id, cheetah, action, cam, date, start_frame, end_frame):
    ''' Calculate centers, scales and estimate 2D keypoints using top-down pose-estimator (pose-detect + '''
    
    #Save frames
    imgnames=[]
    video=mmcv.VideoReader(f'data/acino_3d/{date}/{cheetah}/{action}/{cam}')
    for frame_id, cur_frame in enumerate(mmcv.track_iter_progress(video)):
        if frame_id>=start_frame and frame_id<end_frame: 
            imgname=f'{cheetah}'+date.split('_')[1]+(date.split('_')[2])+f'_{action}.{cam_id}_{count}.jpg'
            mmcv.imwrite(video[frame_id],f'data/acino_3d/images/{imgname}')  # save frame as JPEG file      
            imgnames.append(imgname)
    imgnames=np.array(imgnames)
    
    #2D Anns
    kp2d=np.array(kp2d,dtype=float).reshape(len(anns2d),20,3)
    
    # calculate bounding boxes,centers,scales
    scale_factor=1.2
    bboxes = np.stack([np.min(kp2d[:, :, 0], axis=1),np.min(kp2d[:, :, 1], axis=1),np.max(kp2d[:, :, 0], axis=1),np.max(kp2d[:, :, 1], axis=1)],axis=1)
    centers = np.stack([(bboxes[:, 0] + bboxes[:, 2]) / 2,(bboxes[:, 1] + bboxes[:, 3]) / 2], axis=1)
    scales = scale_factor * np.max(bboxes[:, 2:] - bboxes[:, :2], axis=1) / 200
    
    #3D Anns
    anns3d=mmcv.load(f'data/acino_3d/{date}/{cheetah}/{action}/fte_pw/fte.pickle')
    kp3d=anns3d['positions']:
    kp3d = np.array(kp3d,dtype=float).reshape(len(anns3d['positions']),20,3)
    kp3d = np.concatenate([kp3d, np.ones((len(kp3d), 20, 1))],
                                axis=2)
    
    assert len(imgnames)==kp3d.shape[0]
    assert kp3d.shape[0]==kp2d.shape[0]
    
    return imgnames,centers,scales,kp2d,kp3d

In [18]:
start_frame=13
end_frame=213
date='2019_03_07'
video=mmcv.VideoReader(f'data/acino_3d/2019_03_07/menya/flick/cam1.mp4')
imgnames=[]
for frame_id, cur_frame in enumerate(mmcv.track_iter_progress(video)):
    if frame_id>=start_frame and frame_id<end_frame: 
        imgname=f'menya'+date.split('_')[1]+(date.split('_')[2])+f'_flick.01_{frame_id}.jpg'
        mmcv.image.imwrite(video[frame_id],f'data/acino_3d/2019_03_07/menya/flick/test/{imgname}')  # save frame as JPEG file      
        imgnames.append(imgname)
        

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 576/576, 23.9 task/s, elapsed: 24s, ETA:     0s


In [12]:
#Cameras
cameras={}
for date in os.listdir('data/acino_3d'):
    if date.split('_')[0]=='2019':
        #Cameras
        if osp.exists(f'data/acino_3d/{date}/extrinsic_calib/6_cam_scene.json'):
            calib=f'data/acino_3d/{date}/extrinsic_calib/6_cam_scene.json'
            n=6 # cam scene
            add_cameras(n,calib,cameras)

out_file='data/acino_3d/annotations/cameras.pkl'
with open(out_file, 'wb') as fout:
    pickle.dump(cameras, fout)
print(f'Camera parameters have been written to "{out_file}".\n')

Camera parameters have been written to "data/acino_3d/annotations/cameras.pkl".



In [11]:
det_model = init_detector(
pose_model = init_pose_model(
    
imgnames_all_train=[]
kps3d_all_train=[]

centers_all_train_dlc=[]
scales_all_train_dlc=[]
kps2d_all_train_dlc=[]

centers_all_train_res50=[]
scales_all_train_res50=[]
kps2d_all_train_res50=[]
'''-----------------------'''
imgnames_all_test=[]
kps3d_all_test=[]

centers_all_test_dlc=[]
scales_all_test_dlc=[]
kps2d_all_test_dlc=[]

centers_all_test_res50=[]
scales_all_test_res50=[]
kps2d_all_test_res50=[]

test_count=0
train_count=0
mode='train'

for date in os.listdir('data/acino_3d'):
    
    if date.split('_')[0]=='2019':
        
        #Images
        if not osp.exists(f'data/acino_3d/{date}/processed.txt'):
            for cheetah in os.listdir(f'data/acino_3d/{date}'):
                if cheetah != 'extrinsic_calib':
                    # mmcv.mkdir_or_exist(f'{cheetah}')

                    for action in os.listdir(f'data/acino_3d/{date}/{cheetah}'):
                          
                        reconstr_params=mmcv.load(f'data/acino_3d/{date}/{cheetah}/{action}/fte_pw/reconstruction_params.json')
                        start_frame=reconstr_params['start_frame']
                        end_frame=reconstr_params['end_frame']

                        for cam in os.listdir(f'data/acino_3d/{date}/{cheetah}/{action}'):
                            
                            if cam.endswith('.mp4'):

                                cam_name=cam.split('.')[0]
                                cam_id=date.replace('_','')+'0'+cam_name[-1]
                               
                                imgnames,centers,scales,kps2d,kps3d = get_anns(cam_id, cheetah, action, cam, date, start_frame, end_frame)
                                
                                if mode=='train':
                                    imgnames_all_train.append(imgnames)
                                    centers_all_train.append(centers)
                                    scales_all_train.append(scales)
                                    kps2d_all_train.append(kps2d)
                                    kps3d_all_train.append(kps3d)
                                    print(f'{date}, {mode}, {cheetah}, {action}, {cam}. Train: {train_count}')
                                    train_count+=1
                                    
                                elif mode=='test':
                                    imgnames_all_test.append(imgnames)
                                    centers_all_test.append(centers)
                                    scales_all_test.append(scales)
                                    kps2d_all_test.append(kps2d)
                                    kps3d_all_test.append(kps3d)
                                    test_count+=1
                                    print(f'{date}, {mode}, {cheetah}, {action}, {cam}. Test: {test_count}')
                                    mode='train'
                                    continue
                                    
                                if train_count%6==0:
                                    mode='test'
                                
            # with open(f'data/acino_3d/{date}/processed.txt','w') as processed:
            #     processed.write(f'{date} processed')

imgnames_all_train = np.concatenate(imgnames_all_train)
centers_all_train = np.concatenate(centers_all_train)
scales_all_train = np.concatenate(scales_all_train)
kps2d_all_train = np.concatenate(kps2d_all_train)
kps3d_all_train = np.concatenate(kps3d_all_train)
np.savez(file=f'data/acino_3d/annotations/acino3d_train.npz',imgname=imgnames_all_train,center=centers_all_train,scale=scales_all_train,part=kps2d_all_train,S=kps3d_all_train)
print(f"Saved: train")

kps_3d_all_train = kps3d_all_train[..., :3]  # remove visibility
mean_3d, std_3d = _get_pose_stats(kps_3d_all_train)

kps_2d_all_train = kps2d_all_train[..., :2]  # remove visibility
mean_2d, std_2d = _get_pose_stats(kps_2d_all_train)

# centered around root
# the root keypoint is 0-index
kps_3d_rel = kps_3d_all_train[..., 1:, :] - kps_3d_all_train[..., :1, :]
mean_3d_rel, std_3d_rel = _get_pose_stats(kps_3d_rel)

kps_2d_rel = kps_2d_all_train[..., 1:, :] - kps_2d_all_train[..., :1, :]
mean_2d_rel, std_2d_rel = _get_pose_stats(kps_2d_rel)

stats = {
    'joint3d_stats': {
        'mean': mean_3d,
        'std': std_3d
    },
    'joint2d_stats': {
        'mean': mean_2d,
        'std': std_2d
    },
    'joint3d_rel_stats': {
        'mean': mean_3d_rel,
        'std': std_3d_rel
    },
    'joint2d_rel_stats': {
        'mean': mean_2d_rel,
        'std': std_2d_rel
    }
}

for name, stat_dict in stats.items():
    out_file = f'data/acino_3d/annotations/{name}.pkl'
    with open(out_file, 'wb') as f:
        pickle.dump(stat_dict, f)
    print(f'Create statistic data file: {out_file}')

imgnames_all_test = np.concatenate(imgnames_all_test)
centers_all_test = np.concatenate(centers_all_test)
scales_all_test = np.concatenate(scales_all_test)
kps2d_all_test = np.concatenate(kps2d_all_test)
kps3d_all_test = np.concatenate(kps3d_all_test)
np.savez(file=f'data/acino_3d/annotations/acino3d_test.npz',imgname=imgnames_all_test,center=centers_all_test,scale=scales_all_test,part=kps2d_all_test,S=kps3d_all_test)
print(f"Saved test")


2019_03_05, train, jules, run, cam6.mp4. Train: 0
2019_03_05, train, jules, run, cam2.mp4. Train: 1
2019_03_05, train, jules, run, cam1.mp4. Train: 2
2019_03_05, train, jules, run, cam3.mp4. Train: 3
2019_03_05, train, jules, flick, cam6.mp4. Train: 4
2019_03_05, train, jules, flick, cam2.mp4. Train: 5
2019_03_05, test, jules, flick, cam1.mp4. Test: 1
2019_03_05, train, jules, flick, cam3.mp4. Train: 6
2019_03_05, train, lily, run, cam6.mp4. Train: 7
2019_03_05, train, lily, run, cam5.mp4. Train: 8
2019_03_05, train, lily, run, cam4.mp4. Train: 9
2019_03_05, train, lily, run, cam2.mp4. Train: 10
2019_03_05, train, lily, run, cam1.mp4. Train: 11
2019_03_05, test, lily, run, cam3.mp4. Test: 2
2019_03_05, train, lily, flick, cam6.mp4. Train: 12
2019_03_05, train, lily, flick, cam5.mp4. Train: 13
2019_03_05, train, lily, flick, cam4.mp4. Train: 14
2019_03_05, train, lily, flick, cam2.mp4. Train: 15
2019_03_05, train, lily, flick, cam1.mp4. Train: 16
2019_03_05, train, lily, flick, cam3.mp4

In [16]:
train=np.load('data/acino_3d/annotations/acino3d_train.npz')
print(len(train['imgname']))
test=np.load('data/acino_3d/annotations/acino3d_test.npz')
len(test['imgname'])

13822


2268

#### Train PoseLifter model:

In [2]:
cfg = Config.fromfile('configs/body/3d_kpt_sview_rgb_vid/video_pose_lift/h36m/videopose3d_acino_27frames_fullconv_supervised.py')
# For new configs:

cfg.work_dir='work_dirs/videopose3d_acino_27frames_epoch500'    
# create work_dir
mmcv.mkdir_or_exist(cfg.work_dir)
                                  
# Config Parameters:
cfg.total_epochs=500

cfg.resume_from=None # the checkpoint file to resume from

cfg.gpu_ids=range(1)
cfg.workflow=[('train',1)]

cfg.log_name = time.strftime('%d-%m-', time.localtime())+"ep"+str(cfg.total_epochs)
cfg.log_file = os.path.join(cfg.work_dir, f'{cfg.log_name}.log')

In [ ]:
# Train from config file
autoscale_lr = False # automatically scale lr with the number of gpus
launcher = 'none' # Job launcher. ['none', 'pytorch', 'slurm', 'mpi']
deterministic = False # Whether to set deterministic options for CUDNN backend.
diff_seed = False # Whether or not set different seeds for different ranks
seed = 0 

if autoscale_lr:
    # apply the linear scaling rule (https://arxiv.org/abs/1706.02677)
    cfg.optimizer['lr'] = cfg.optimizer['lr'] * len(cfg.gpu_ids) / 8

if launcher == 'none':
    distributed = False
    if len(cfg.gpu_ids) > 1:
        warnings.warn(
            f'We treat {cfg.gpu_ids} as gpu-ids, and reset to '
            f'{cfg.gpu_ids[0:1]} as gpu-ids to avoid potential error in '
            'non-distribute training time.')
        cfg.gpu_ids = cfg.gpu_ids[0:1]
else:
    distributed = True
    init_dist(launcher, **cfg.dist_params)
    # re-set gpu_ids with distributed training mode
    _, world_size = get_dist_info()
    cfg.gpu_ids = range(world_size)

# set multi-process settings
setup_multi_processes(cfg)

# init the logger before other steps
logger = get_root_logger(log_file=cfg.log_file, log_level=cfg.log_level)

# init the meta dict to record some important information such as
# environment info and seed, which will be logged
meta = dict()

# log env info
env_info_dict = collect_env()
env_info = '\n'.join([(f'{k}: {v}') for k, v in env_info_dict.items()])
dash_line = '-' * 60 + '\n'
# logger.info('Environment info:\n' + dash_line + env_info + '\n' +
#             dash_line)
meta['env_info'] = env_info

# log some basic info
logger.info(f'Distributed training: {distributed}')
# logger.info(f'Config:\n{cfg.pretty_text}')

# set random seeds
seed = init_random_seed(seed)
seed = seed + dist.get_rank() if diff_seed else seed
logger.info(f'Set random seed to {seed}, '
            f'deterministic: {deterministic}')
set_random_seed(seed, deterministic=deterministic)
cfg.seed = seed
meta['seed'] = seed

# build dataset
train_set = [build_dataset(cfg.data.train)]

# build model
model = build_posenet(cfg.model)

if len(cfg.workflow) == 2:
    val_dataset = copy.deepcopy(cfg.data.val)
    val_dataset.pipeline = cfg.data.train.pipeline
    train_set.append(build_dataset(val_dataset))

if cfg.checkpoint_config is not None:
    # save mmpose version, config file content
    # checkpoints as meta data
    cfg.checkpoint_config.meta = dict(
        mmpose_version=__version__ + get_git_hash(digits=7),
        # config=cfg.pretty_text,
    )

timestamp = time.strftime('%d-%m-', time.localtime())+"ep"+str(cfg.total_epochs)
print("LINE")
# train model
train_model(
    model,
    train_set, 
    cfg, 
    distributed=distributed, 
    validate=True, 
    timestamp=timestamp,
    meta=meta)

In [ ]:
!python demo/body3d_two_stage_video_demo.py MMDet/configs/faster_rcnn_r50_fpn_2x_acino.py MMDet/epoch_4.pth configs/animal/2d_kpt_sview_rgb_img/topdown_heatmap/acino/res50_acino_256x256.py work_dirs/res50_acino_256x256_epoch150/.pth configs/body/3d_kpt_sview_rgb_vid/video_pose_lift/h36m/videopose3d_acino_27frames_fullconv_supervised.py work_dirs/videopose3d_acino_27frames_epoch160/latest.pth --video-path cam3ph.mp4 --out-video-root cam3ph3D

In [ ]:
test=np.load('data/acino_3d/annotations/acino3d_test.npz')
train=np.load('data/acino_3d/annotations/acino3d_train.npz')
all_centers=np.concatenate((test['center'],train['center']),axis=0)
center_avg=np.average(all_centers,axis=0)
print(center_avg)

all_scales=np.concatenate((test['scale'],train['scale']),axis=0)
# print(all_scales)
scale_avg=np.average(all_scales,axis=0)
scale_avg

In [4]:
person_det_model = init_detector('MMDet/configs/faster_rcnn_r50_fpn_2x_acino.py', 'MMDet/epoch_4.pth')
pose_det_model=init_pose_model('configs/animal/2d_kpt_sview_rgb_img/topdown_heatmap/acino/res50_acino_256x256.py','work_dirs/res50_acino_256x256_epoch150/latest.pth')

pose_det_dataset = pose_det_model.cfg.data['test']['type']
# get datasetinfo
dataset_info = pose_det_model.cfg.data['test'].get('dataset_info', None)
dataset_info=DatasetInfo(dataset_info)
# print(dataset_info.skeleton)

video = mmcv.VideoReader('cam3.mp4')

return_heatmap=False
output_layer_names = None
use_multi_frames=False
rebase_keypoint_height=True
num_instances=-1
pose_det_results_list = []
next_id = 0
pose_det_results = []
save_out_video=True

print("\n2D Detection and Pose Estimation:")
for frame_id, cur_frame in enumerate(mmcv.track_iter_progress(video)):
    
    pose_det_results_last = pose_det_results
    
    # get the detection results of current frame
    # the resulting box is (x1, y1, x2, y2)
    mmdet_results = inference_detector(person_det_model, cur_frame)

    # keep the person class bounding boxes.
    person_det_results = process_mmdet_results(mmdet_results, 1)

    # if use_multi_frames:
    #     frames = collect_multi_frames(video, frame_id, indices,
    #                                   args.online)

    # test a single image, with a list of bboxes.
    pose_det_results, _ = inference_top_down_pose_model(
        pose_det_model,
        cur_frame,
        person_det_results,
        format='xyxy',
        dataset=pose_det_dataset,
        dataset_info=dataset_info,
        return_heatmap=return_heatmap,
        outputs=output_layer_names)
    
    pose_det_results, next_id = get_track_id(
            pose_det_results,
            pose_det_results_last,
            next_id)
    
    pose_det_results_list.append(copy.deepcopy(pose_det_results))

    # show the results
    # if frame_id%20==0:
    #     out=f'menyacam3/{frame_id}.jpg'  # save frame as JPEG file
    #     # print(pose_det_results)
    # else:
    #     out=None
      
    vis_frame = vis_pose_result(
        pose_det_model,
        cur_frame,
        pose_det_results,
        dataset=pose_det_dataset,
        dataset_info=dataset_info,
        show=False,
        out_file=None)
        # kpt_score_thr=0.003
    
pose_lift_model=init_pose_model('configs/body/3d_kpt_sview_rgb_vid/video_pose_lift/h36m/videopose3d_acino_27frames_fullconv_supervised.py',
                                'work_dirs/videopose3d_acino_27frames_epoch500/latest.pth')
pose_lift_dataset = pose_lift_model.cfg.data['test']['type']

if save_out_video:
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = video.fps
    writer = None

for pose_det_results in pose_det_results_list:
    for res in pose_det_results:
        keypoints = res['keypoints']
        keypoints_new = np.zeros((20, keypoints.shape[1]), dtype=keypoints.dtype)
        keypoints_new=keypoints[[2,1,0,3,
                                23,4,18,17,
                                8,9,19,
                                5,6,20,
                                14,15,21,
                                11,12,22]] 
        res['keypoints']=keypoints_new
        
if hasattr(pose_lift_model.cfg, 'test_data_cfg'):
    data_cfg = pose_lift_model.cfg.test_data_cfg
else:
    data_cfg = pose_lift_model.cfg.data_cfg

smoother=Smoother('configs/_base_/filters/one_euro.py',keypoint_key='keypoints',keypoint_dim=2)
# if i==80:
#         print(pose_results_2d)
#         print()
        
#         print(pose_lift_results[0])
pose_lift_dataset_info = pose_lift_model.cfg.data['test'].get('dataset_info', None)
pose_lift_dataset_info = DatasetInfo(pose_lift_dataset_info)

print("\nPose Lifting:")
for i, pose_det_results in enumerate(mmcv.track_iter_progress(pose_det_results_list)): #Pose det results: iterate over each frame
    # extract and pad input pose2d sequence
    pose_results_2d = extract_pose_sequence( # returns 2D kp and BBox for 27 frames. len = 27
        pose_det_results_list,
        frame_idx=i,
        causal=data_cfg.causal,
        seq_len=data_cfg.seq_len,
        step=data_cfg.seq_frame_interval)

    # # smooth 2d results
    # if smoother:
    #     pose_results_2d = smoother.smooth(pose_results_2d)

    # 2D-to-3D pose lifting
    pose_lift_results = inference_pose_lifter_model( #'keypoints': 2D keypoints for 27 frames, 'keypoints_3d': 3D keypoints for target frame
        pose_lift_model,
        pose_results_2d=pose_results_2d,
        dataset=pose_lift_dataset,
        dataset_info=pose_lift_dataset_info,
        with_track_id=True,
        image_size=video.resolution,
        norm_pose_2d=False)
    # print(pose_lift_results) #Norm = true
    
              
    pose_lift_results_vis = []
    for idx, res in enumerate(pose_lift_results):
        keypoints_3d = res['keypoints_3d']
        # exchange y,z-axis, and then reverse the direction of x,z-axis
        # keypoints_3d = keypoints_3d[..., [0, 2, 1]]
        # keypoints_3d[..., 0] = -keypoints_3d[..., 0]
        # keypoints_3d[..., 2] = -keypoints_3d[..., 2]
    #     # rebase height (z-axis)
        if rebase_keypoint_height:
            keypoints_3d[..., 2] -= np.min(
                keypoints_3d[..., 2], axis=-1, keepdims=True)
        res['keypoints_3d'] = keypoints_3d
    #     # add title
        det_res = pose_det_results[idx]
        instance_id = det_res['track_id']
        res['title'] = f'Prediction ({instance_id})'
    #     # only visualize the target frame
        res['keypoints'] = det_res['keypoints']
        res['bbox'] = det_res['bbox']
        res['track_id'] = instance_id
        pose_lift_results_vis.append(res)
    
    # print(len(pose_lift_results_vis))
    # Visualization
    
    if len(pose_lift_results_vis)==1:

        if num_instances < 0:
            num_instances = len(pose_lift_results_vis)
        # if i==80:
        #     out='m/frame80.jpg'
        #     print(pose_lift_results_vis)
        # else:
        #     out=None
            
        img_vis = vis_3d_pose_result(
            pose_lift_model,
            result=pose_lift_results_vis,
            img=video[i],
            dataset=pose_lift_dataset,
            dataset_info=pose_lift_dataset_info,
            out_file=None,
            num_instances=num_instances,
            show=False)

        if save_out_video:
            if writer is None:
                writer = cv2.VideoWriter(
                    osp.join('cam3ph3D',
                             'cam3ph_out.mp4'), fourcc,
                    fps, (img_vis.shape[1], img_vis.shape[0]))
            writer.write(img_vis)

if save_out_video:
    writer.release()

load checkpoint from local path: MMDet/epoch_4.pth
load checkpoint from local path: work_dirs/res50_acino_256x256_epoch150/latest.pth

2D Detection and Pose Estimation:
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 201/201, 6.8 task/s, elapsed: 29s, ETA:     0s
load checkpoint from local path: work_dirs/videopose3d_acino_27frames_epoch500/latest.pth

Pose Lifting:
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 201/201, 9.8 task/s, elapsed: 20s, ETA:     0s


In [ ]:
pose_lift_results_vis

In [ ]:
menya3d['positions'][0]

In [ ]:
pose_lift_results[0]['keypoints_3d']

In [ ]:
print(pose_lift_results[0]['bbox']) #Norm = true

In [ ]:
menya3d=mmcv.load('menyacam3/fte.pickle')
# print(menya3d['positions'][0])
print(menya3d['positions'][80-69])
kp3d=[]
for ann in menya3d['positions']:
    spine=ann[4]
    ann=np.delete(ann,4,axis=0)
    ann=np.insert(ann,0,spine,axis=0)
    kp3d.append(ann)

kp3d=np.array(kp3d,dtype=float).reshape(len(menya3d['positions']),20,3)
kp3d = np.concatenate([kp3d, np.ones((len(kp3d), 20, 1))],
                                axis=2)
# print(kp3d[80-69])
# kp3d = kp3d[..., [0, 2, 1]]
# kp3d[..., 0] = -kp3d[..., 0]
# kp3d[..., 2] = -kp3d[..., 2]
# print(menya3d['positions']) # [0] == frame 69 | [11]==frame80

print(kp3d[80-69])

In [ ]:
print(pose_lift_dataset_info.skeleton)

In [ ]:
pose_lift_model=init_pose_model('configs/body/3d_kpt_sview_rgb_vid/video_pose_lift/h36m/videopose3d_acino_27frames_fullconv_supervised.py','work_dirs/videopose3d_acino_27frames_epoch160/latest.pth')
pose_lift_dataset = pose_lift_model.cfg.data['test']['type']
pose_lift_dataset_info = pose_lift_model.cfg.data['test'].get('dataset_info', None)
pose_lift_dataset_info=DatasetInfo(pose_lift_dataset_info)

menya3d=mmcv.load('menyacam3/fte.pickle')
joints=menya3d['positions'][11]
root_idx=4

root = joints[..., root_idx:root_idx + 1, :]
joints = joints - root
# print(joints)
joints = joints[..., [0, 2, 1]]
joints[..., 0] = -joints[..., 0]
joints[..., 2] = -joints[..., 2]

joints=[joints]
# joints=np.concatenate([joints, np.ones((len(joints), 20, 1))],
#                                 axis=2)

result=dict(keypoints_3d=joints[0],track_id=int(0))
result=[result]
vis_3d_pose_result(
            pose_lift_model,
            result=result,
            img=None,#video[i],
            dataset=pose_lift_dataset,
            dataset_info=pose_lift_dataset_info,
            out_file='menyacam3/gt.jpg',
            num_instances=-1,
            show=False)


In [ ]:
menya3d=load_pickle('menyacam3/fte.pickle')
menya3d

In [ ]:
def load_pickle(pickle_file):
    """
    Loads a dictionary from a saved skeleton .pickle file
    """
    with open(pickle_file, 'rb') as handle:
        data = pickle.load(handle)

    return(data)

In [ ]:
camera=mmcv.load('data/acino_3d/annotations/cameras.pkl')


In [ ]:
# data settings
data_root = 'data/acino_3d'
data_cfg = dict(
    num_joints=20,
    seq_len=27,
    seq_frame_interval=1,
    causal=False,
    temporal_padding=True,
    joint_2d_src='gt',
    need_camera_param=True,
    camera_param_file=f'{data_root}/annotations/cameras.pkl',
)

pose_lift_model=init_pose_model('configs/body/3d_kpt_sview_rgb_vid/video_pose_lift/h36m/videopose3d_acino_27frames_fullconv_supervised.py','work_dirs/videopose3d_acino_27frames_epoch160/latest.pth')
pose_lift_dataset = pose_lift_model.cfg.data['test']['type']
pose_lift_dataset_info = pose_lift_model.cfg.data['test'].get('dataset_info', None)
pose_lift_dataset_info=DatasetInfo(pose_lift_dataset_info)
data=np.load('data/acino_3d/annotations/acino3d_train.npz')

if 'scale' in data:
    _scales = data['scale'].astype(np.float32)
else:
    _scales = np.zeros(num_imgs, dtype=np.float32)

if 'center' in data:
    _centers = data['center'].astype(np.float32)
else:
    _centers = np.zeros((num_imgs, 2), dtype=np.float32)
_imgnames = data['imgname']
num_imgs = len(_imgnames)
num_joints=data_cfg['num_joints']
# get 3D pose
if 'S' in data.keys():
    _joints_3d = data['S'].astype(np.float32)
else:
    _joints_3d = np.zeros((num_imgs, num_joints, 4), dtype=np.float32)

# get 2D pose
if 'part' in data.keys():
    _joints_2d = data['part'].astype(np.float32)
else:
    _joints_2d = np.zeros((num_imgs, num_joints, 3), dtype=np.float32)

data_info = {
    'imgnames': _imgnames,
    'joints_3d': _joints_3d,
    'joints_2d': _joints_2d,
    'scales': _scales,
    'centers': _centers,
}

joint_2d_src = data_cfg.get('joint_2d_src', 'gt')
# if joint_2d_src not in self.SUPPORTED_JOINT_2D_SRC:
#     raise ValueError(
#         f'Unsupported joint_2d_src "{self.joint_2d_src}". '
#         f'Supported options are {self.SUPPORTED_JOINT_2D_SRC}')

joint_2d_det_file = data_cfg.get('joint_2d_det_file', None)

need_camera_param = data_cfg.get('need_camera_param', False)

actions = data_cfg.get('actions', '_all_')
actions = set(actions if isinstance(actions, (list, tuple)) else [actions])
subjects = data_cfg.get('subjects', '_all_')

def _parse_h36m_imgname(imgname):
    subj, rest = osp.basename(imgname).split('_', 1)
    action, rest = rest.split('.', 1)
    camera, rest = rest.split('_', 1)
    return subj,action,camera

video_frames = defaultdict(list)
for idx, imgname in enumerate(data_info['imgnames']):
    subj, action, camera = _parse_h36m_imgname(imgname)

    if '_all_' not in actions and action not in actions:
        continue

    if '_all_' not in subjects and subj not in subjects:
        continue

    video_frames[(subj, action, camera)].append(idx)
    
video_frames

In [20]:
# build sample indices
seq_frame_interval=1
seq_len=27
subset=1
sample_indices = []
_len = (seq_len - 1) * seq_frame_interval + 1
_step = seq_frame_interval
for _, _indices in sorted(video_frames.items()):
    n_frame = len(_indices)

            # Pad the sequence so that every frame in the sequence will be
    # predicted.
    frames_left = (seq_len - 1) // 2
    frames_right = frames_left
    for i in range(n_frame):
        pad_left = max(0, frames_left - i // _step)
        pad_right = max(0,
                        frames_right - (n_frame - 1 - i) // _step)
        start = max(i % _step, i - frames_left * _step)
        end = min(n_frame - (n_frame - 1 - i) % _step,
                  i + frames_right * _step + 1)
        sample_indices.append([_indices[0]] * pad_left +
                              _indices[start:end:_step] +
                              [_indices[-1]] * pad_right)
 

# reduce dataset size if self.subset < 1
assert 0 < subset <= 1
subset_size = int(len(sample_indices) *subset)
start = np.random.randint(0, len(sample_indices) - subset_size + 1)
end = start + subset_size

In [ ]:
sample_indices[start:end]